In [2]:
from ipynb.fs.full.clean import clean_relevant_events
import pandas as pd

def create_cohort_table(cur, file_path, conn):
    commands = ['''DROP TABLE IF EXISTS model_a_cohort''',
                '''CREATE TABLE model_a_cohort (
                  identifier VARCHAR(50),
                  subject_id VARCHAR(50),
                  hadm_id VARCHAR(50),
                  admittime TIMESTAMP,
                  icu_time TIMESTAMP,
                  target_time TIMESTAMP
                )''']
    for command in commands:
        cur.execute(command)
    sql = "COPY model_a_cohort FROM STDIN DELIMITER ',' CSV HEADER"
    cur.copy_expert(sql, open(file_path, "r"))
    conn.commit()
    print("done creating cohort table for model a.")


def create_relevent_events_table(cur, conn):
    commands = ["DROP TABLE IF EXISTS _relevantFeatures",
                '''create table _relevantFeatures(
                item_id INT,
                _table TEXT	)'''
                ]
    for command in commands:
        cur.execute(command)
    sql = "INSERT INTO _relevantFeatures(item_id , _table) VALUES(%s, %s)"
    feat_list = [
        # Complete Blood Count:
        (50889, 'labevents'),  # C-Reactive Protein
        (51256, 'labevents'),  # Neturophils
        (51279, 'labevents'),  # Red Blood Cells
        (50811, 'labevents'),  # Hemoglobin
        (51221, 'labevents'),  # Hematocrit
        (51250, 'labevents'),  # MCV
        (51248, 'labevents'),  # MCH
        (51249, 'labevents'),  # MCHC
        (51277, 'labevents'),  # RDW
        (51244, 'labevents'),  # Lymphocytes
        (51254, 'labevents'),  # Monocytes
        (51200, 'labevents'),  # Eosinophils
        (51146, 'labevents'),  # Basophils
        (51265, 'labevents'),  # Platelet Count
        # Basic Metabolic Panel:
        (50971, 'labevents'),  # Potassium
        (50983, 'labevents'),  # Sodium
        (50912, 'labevents'),  # Creatinine
        (50902, 'labevents'),  # Chloride
        (51006, 'labevents'),  # Urea Nitrogen
        (50882, 'labevents'),  # Bicarbonate
        (50868, 'labevents'),  # Anion Gap
        (50931, 'labevents'),  # Glucose
        (50960, 'labevents'),  # Magnesium
        (50893, 'labevents'),  # Calcium, Total
        (50970, 'labevents'),  # Phosphate
        (50820, 'labevents'),  # pH
        # Blood Gases:
        (50802, 'labevents'),  # Base Excess
        (50804, 'labevents'),  # Calculated Total CO2
        (50821, 'labevents'),  # pO2
        (50818, 'labevents'),  # pCO2
        (50813, 'labevents'),  # Lactate
        # Cauglation Panel:
        (51275, 'labevents'),  # PTT
        (51237, 'labevents'),  # INR(PT)
        (51274, 'labevents'),  # PT
        # Liver function:
        (50885, 'labevents'),  # Bilirubin, Total
        (50976, 'labevents'),  # Protein, Total
        (50862, 'labevents'),  # Albumin
        (50863, 'labevents'),  # Alkaline Phosphatase
        (50861, 'labevents'),  # Alanine Aminotransferase(ALT)
        (50878, 'labevents'),  # Asparate Aminotransferase(AST)
        # Vital signs:
        (223762, 'chartevents'),  # Temp C, metavision "Temperature Celsius"
        (676, 'chartevents'),  # Temp C, careVue "Temperature C"
        (220045, 'chartevents'),  # HearRate, MetaVision "Heart Rate"
        (211, 'chartevents'),  # HearRate, CareVue "Heart Rate"
        (220277, 'chartevents'),  # SpO2 saturation, metavision "O2 saturation pulseoxymetry"
        (646, 'chartevents'),  # SpO2 saturation, CareVue "SpO2"
        (220181, 'chartevents'),  # Non Invasive Blod Pressure(mean), MetaVision "Non Invasive Blod Pressure mean"
        (456, 'chartevents'),  # Non Invasive Blod Pressure(mean), CareVue "NBP Mean"
        (220179, 'chartevents'),  # Non Invasive Blood Pressure systolic, metavision
        (455, 'chartevents'),  # NBP[Systolic], CareVue
        (220180, 'chartevents'),  # Non InvasiveBlood Pressure diastolic, metavision
        (8441, 'chartevents'),  # NBP[Diastolic], CareVue
        (220050, 'chartevents'),  # Arterial Blood Pressure systolic, metavision
        (6, 'chartevents'),  # ABP[Systolic], CareVue
        (220051, 'chartevents'),  # Arterial Blood Pressure diastolic, MetaVision
        (8364, 'chartevents'),  # ABP[Diastolic], CareVue
        (220052, 'chartevents')  # Arterial Blood Pressure mean, metavision
    ]
    cur.executemany(sql, feat_list)
    
    print("start creating relevent_events.")
    
    commands = [  # Create tables of all relevant rows from labevents for the cohort:
        "DROP TABLE IF EXISTS relevant_labevents_for_cohort",
        '''CREATE TABLE relevant_labevents_for_cohort as (
                select subject_id||'-'||hadm_id as identifier, subject_id, hadm_id, itemid, charttime, valuenum, label
                from mimiciii.labevents 
                join (select itemid, label from mimiciii.d_labitems) as t1 using (itemid)
                where subject_id||'-'||hadm_id in (select identifier from model_a_cohort) 
                AND itemid in (select item_id from _relevantFeatures where _table='labevents')
                )''',
        # Create tables of all relevant rows from chartevents for the cohort:
        "DROP TABLE IF EXISTS relevant_chartevents_for_cohort",
        '''CREATE TABLE relevant_chartevents_for_cohort as (
                    select subject_id||'-'||hadm_id as identifier, subject_id, hadm_id, itemid, charttime, valuenum, label
                    from mimiciii.chartevents 
                    join (select itemid, label from mimiciii.d_items) as t1 using (itemid)
                    where subject_id||'-'||hadm_id in (select identifier from model_a_cohort) 
                    AND itemid in (select item_id from _relevantFeatures where _table='chartevents')
                )''',
        # Create a unified table of feature from the tables created above:
        "DROP TABLE IF EXISTS all_relevant_lab_features",
        '''CREATE TABLE all_relevant_lab_features as (
                    select * from 
                    relevant_chartevents_for_cohort 
                    union 
                    (select * from relevant_labevents_for_cohort)
                )''',
        # Create a table of relevant events (features) received near when the target (culture) was received:
        "DROP TABLE IF EXISTS relevant_events",
        '''CREATE TABLE relevant_events as(
                SELECT *,
                        date_part('year', admittime) - date_part('year', dob) as estimated_age,
                        round(CAST((extract(epoch from target_time - all_relevant_lab_features.charttime) / 3600.0) as numeric),2) as hours_from_charttime_to_targettime,
                        round(CAST((extract(epoch from charttime - admittime) / 3600.0 ) as numeric),2) as hours_from_admittime_to_charttime,
                        round(CAST((extract(epoch from target_time - admittime) / 3600.0) as numeric),2) as hours_from_admittime_to_targettime
                FROM 
                    all_relevant_lab_features			
                    INNER JOIN (select identifier, target_time, admittime from model_a_cohort) _tmp2 using (identifier)
                    INNER JOIN (select subject_id, gender, dob from mimiciii.patients 
                                    where subject_id in (select CAST (subject_id as INTEGER) from model_a_cohort)) as t3 	
                                    using (subject_id)
                WHERE identifier in (select identifier from model_a_cohort)
                AND (extract(epoch from target_time - all_relevant_lab_features.charttime)) > 0
                )'''
    ]
    for command in commands:
        cur.execute(command)
    conn.commit()
    print("done creating relevent_events.")
    updated_relevent_events(cur, conn)


def updated_relevent_events(cur, conn):
    commands = ['''UPDATE relevant_events
                SET label = 'Temperature Celsius' 
                WHERE label in ('Temperature C', 'Temperature Celsius')''',

                '''UPDATE relevant_events
                SET label = 'SpO2' 
                WHERE label in ('O2 saturation pulseoxymetry')''',

                '''UPDATE relevant_events
                SET label = 'Non Invasive Blood Pressure mean' 
                WHERE label in ('NBP Mean', 'Non Invasive Blod Pressure mean')''',

                '''UPDATE relevant_events
                SET label = 'Non Invasive Blood Pressure systolic' 
                WHERE label in ('NBP [Systolic]')''',

                '''UPDATE relevant_events
                SET label = 'Non Invasive Blood Pressure diastolic' 
                WHERE label in ('NBP [Diastolic]')''',

                '''UPDATE relevant_events
                SET label = 'Arterial Blood Pressure systolic' 
                WHERE label in ('ABP [Systolic]')''',

                '''UPDATE relevant_events
                SET label = 'Arterial Blood Pressure diastolic' 
                WHERE label in ('ABP [Diastolic]')''',

                '''ALTER TABLE relevant_events
                RENAME COLUMN valuenum TO value'''
                ]
    for command in commands:
        cur.execute(command)
    conn.commit()
    print("done updating relevent_events.")


def create_updated_cohort(cur, conn):
    # exclude from the cohort the patients that have no records in relevant_events table:
    commands = ["DROP TABLE IF EXISTS model_a_cohort_updated",
                '''CREATE TABLE model_a_cohort_updated as (
                select * 
                from model_a_cohort
                where identifier in (select distinct identifier from relevant_events)
                )'''
                ]
    for command in commands:
        cur.execute(command)
    conn.commit()
    print("done creating model_a_cohort_updated.")


def create_antibiotics_table(cur, conn):
    commands = ["DROP TABLE IF EXISTS antibiotics",
                '''CREATE TABLE antibiotics as (
                select subject_id||'-'||hadm_id as identifier, subject_id, hadm_id, icustay_id, 
                    case when (starttime > endtime) then endtime else starttime end as start_date, 
                    case when (starttime > endtime) then starttime else endtime end as end_date,
                    label as order_name, originalamount as amount, amountuom, originalrate, patientweight, target_time, 
                    case when (starttime > endtime) then round(CAST((extract(epoch from target_time - endtime) / 3600.0) as numeric),2) 
                        else round(CAST((extract(epoch from target_time - starttime) / 3600.0) as numeric),2) 
                        end as hours_from_starttime_to_targettime
                    from INPUTEVENTS_MV as a
                    inner join D_ITEMS as b using (itemid)
                    inner join (select identifier, target_time from model_a_cohort_updated) as c 
                        on a.subject_id||'-'||a.hadm_id = c.identifier
                    where lower(ordercategoryname) like '%anti%'
                    and label not in ('Dextrose 5%', 'Solution', 'NaCl 0.9%')
                    and originalamount > 0
                    and subject_id||'-'||hadm_id in (select identifier from model_a_cohort_updated)
                    and (extract(epoch from target_time - starttime)) > 0
                    order by identifier, starttime
                )'''
                ]
    for command in commands:
        cur.execute(command)
    conn.commit()
    print("done creating antibiotics.")


def create_pressor_sedatives_meds_table(cur, conn):
    commands = ["DROP TABLE IF EXISTS pressor_sedatives",
                '''CREATE TABLE pressor_sedatives as (
                with tmp as (
                select subject_id||'-'||hadm_id as identifier, subject_id ,hadm_id ,icustay_id, starttime as start_date, endtime as end_date,
                case when label = 'Midazolam (Versed)' then 'Midazolam' else label end as order_name, 
                abs(COALESCE(COALESCE(rate,originalrate),amount)) as rate, amount, target_time,
                round(CAST((extract(epoch from target_time - starttime) / 3600.0) as numeric),2) as hours_from_starttime_to_targettime
                from inputevents_mv as a
                inner join D_ITEMS as b using (itemid)
                inner join (select identifier, target_time from model_a_cohort_updated) as c 
                        on a.subject_id||'-'||a.hadm_id = c.identifier
                where b.category  in ('Medications')
                and b.label in ('Epinephrine','Dopamine','Midazolam (Versed)','Fentanyl','Phenylephrine','Norepinephrine','Propofol','Vasopressin','Morphine Sulfate')
                and a.statusdescription not in ('Rewritten')
                and subject_id||'-'||hadm_id in (select identifier from model_a_cohort_updated)
                and (extract(epoch from target_time - starttime)) > 0
                order by identifier, starttime
                )
                select *
                from tmp
                where rate>0
                )'''
                ]
    for command in commands:
        cur.execute(command)
    conn.commit()
    print("done creating pressor_sedatives.")


def create_output_liquid_table(cur, conn):
    commands = ["DROP TABLE IF EXISTS output_liquid_table",
                '''CREATE TABLE output_liquid_table as (
                    select subject_id||'-'||hadm_id as identifier, subject_id, hadm_id ,icustay_id, charttime, value as value, label as parameter_name, target_time,
                    round(CAST((extract(epoch from target_time - charttime) / 3600.0) as numeric),2) as hours_from_charttime_to_targettime
                    from OUTPUTEVENTS as a
                    inner join D_ITEMS as b using (itemid) 
                    inner join (select identifier, target_time from model_a_cohort_updated) as d
                        on a.subject_id||'-'||a.hadm_id = d.identifier
                    where UNITNAME = 'mL'
                    and LABEL in ('R Ureteral Stent','L Ureteral Stent','Foley','Void','R Nephrostomy', 'L Nephrostomy','Straight Cath')
                    and (extract(epoch from target_time - charttime)) > 0
                    order by identifier, charttime
                )'''
                ]
    for command in commands:
        cur.execute(command)
    conn.commit()
    print("done creating output_liquid_table.")


def create_input_liquid_table(cur, conn):
    commands = ["DROP TABLE IF EXISTS input_liquid_table",
                '''CREATE TABLE input_liquid_table as (
                    select subject_id||'-'||hadm_id as identifier, subject_id, hadm_id ,icustay_id, starttime as start_date, endtime as end_date, 
                    label as order_name, category, target_time, amount, originalamount ,amountuom, originalrate, patientweight,
                    round(CAST((extract(epoch from target_time - starttime) / 3600.0) as numeric),2) as hours_from_starttime_to_targettime
                    from INPUTEVENTS_MV a
                    inner join D_ITEMS as b using (itemid) 
                    inner join (select identifier, target_time from model_a_cohort_updated) as d
                        on a.subject_id||'-'||a.hadm_id = d.identifier
                    where UNITNAME = 'mL'
                    and CATEGORY in ('Fluids - Other (Not In Use)','Fluids/Intake','Blood Products/Colloids')
                    and (extract(epoch from target_time - starttime)) > 0
                    order by identifier, starttime
                )'''
                ]
    for command in commands:
        cur.execute(command)
    conn.commit()
    print("done creating input_liquid_table.")


def create_gcs_score_table(cur, conn):
    commands = ["DROP TABLE IF EXISTS gcs_score",
                '''CREATE TABLE gcs_score as (
                    select subject_id||'-'||hadm_id as identifier, subject_id, hadm_id ,icustay_id, charttime, label, 
                    valuenum as score, value as description, target_time,
                    round(CAST((extract(epoch from target_time - charttime) / 3600.0) as numeric),2) as hours_from_charttime_to_targettime
                    from chartevents as a 
                    inner join D_ITEMS as b using (itemid)
                    inner join (select identifier, target_time from model_a_cohort_updated) as c 
                        on a.subject_id||'-'||a.hadm_id = c.identifier
                    where itemid in (220739,223900,223901, 198) 
                    and subject_id||'-'||hadm_id in (select identifier from model_a_cohort_updated)
                    and (extract(epoch from target_time - charttime)) > 0
                    and valuenum is not null
                    order by identifier, charttime
                )
                -- 220739 GCS - Eye Opening
                -- 223900 GCS - Verbal Response
                -- 223901 GCS - Motor Response
                '''
                ]
    for command in commands:
        cur.execute(command)
    conn.commit()
    print("done creating gcs_score.")


def create_bmi_table(cur, conn):
    commands = [  # create table for weight:
        "DROP TABLE IF EXISTS weight",
        '''CREATE TABLE weight as (
                select identifier, subject_id, hadm_id, admit_weight_kg from 
                (
                    select *, ROW_NUMBER() OVER (PARTITION BY identifier ORDER BY charttime) as rn
                    from
                        (select identifier, c.subject_id, c.hadm_id, weight_kg as admit_weight_kg, charttime, itemid
                        from (select identifier, subject_id, hadm_id from model_a_cohort_updated) as c
                        left join (select subject_id, hadm_id, itemid, charttime, valuenum as weight_kg from chartevents 
                              where itemid in(226512, 224639, 763)) as a 
                            on a.subject_id||'-'||a.hadm_id = c.identifier
                        order by identifier, charttime
                        ) as a 
                ) as b
                where rn = 1
                )
                -- 226512  Admission Weight (Kg) --metavision
                -- 224639  Daily Weight --metavision
                -- 763	   Daily Weight --carevue
                ''',
        # create table for height:
        "DROP TABLE IF EXISTS height",
        '''CREATE TABLE height as (
                select identifier, subject_id, hadm_id, height_cm from 
                (
                    select *, ROW_NUMBER() OVER (PARTITION BY identifier ORDER BY charttime) as rn
                    from
                        (select identifier, c.subject_id, c.hadm_id, a.charttime,
                            case when itemid = 1394 then height*2.54 
                                when itemid = 226730 then height 
                            end as height_cm
                        from (select identifier, subject_id, hadm_id from model_a_cohort_updated) as c
                        left join (select subject_id, hadm_id, itemid, charttime, valuenum as height from chartevents where itemid in(226730,1394)) as a 
                            on a.subject_id||'-'||a.hadm_id = c.identifier
                        order by identifier, charttime
                        ) as a 
                ) as b
                where rn = 1
                and identifier in (select distinct identifier from model_a_cohort_updated)
                )''',
        # combine weight and height in one table:
        "DROP TABLE IF EXISTS bmi",
        '''CREATE TABLE bmi as (
                select * from weight
                left join (select identifier, height_cm from height) as h using (identifier)
                )'''
    ]
    for command in commands:
        cur.execute(command)
    conn.commit()
    print("done creating bmi.")


def create_daily_weight_table(cur, conn):
    commands = ["DROP TABLE IF EXISTS daily_weight",
                '''CREATE TABLE daily_weight as (
                    select subject_id||'-'||hadm_id as identifier, subject_id, hadm_id ,icustay_id, charttime,
                    valuenum as kg, target_time,
                    round(CAST((extract(epoch from target_time - charttime) / 3600.0) as numeric),2) as hours_from_charttime_to_targettime
                    from chartevents as a 
                    inner join (select identifier, target_time from model_a_cohort_updated) as c 
                        on a.subject_id||'-'||a.hadm_id = c.identifier
                    where itemid in (226512, 224639, 763) 
                    and subject_id||'-'||hadm_id in (select identifier from model_a_cohort_updated)
                    and (extract(epoch from target_time - charttime)) > 0
                    order by identifier, charttime
                )'''
                ]
    for command in commands:
        cur.execute(command)
    conn.commit()
    print("done creating daily_weight.")


def create_respiratory_checks_table(cur, conn):
    commands = ["DROP TABLE IF EXISTS respiratory_checks",
                '''CREATE TABLE respiratory_checks as (
                select subject_id||'-'||hadm_id as identifier, subject_id, hadm_id ,icustay_id, itemid, charttime, 
                valuenum as value, valueuom as measure_unit, 
                case when label = 'Insp. Time' then 'Inspiratory Time'
                    when label = 'Inspired O2 Fraction' then 'FIO2'
                    when label = 'PEEP set' then 'PEEP Set'
                    when label = 'PEEP Set' then 'PEEP' 
                    when label = 'Respiratory Rate Set' then 'Respiratory Rate (Set)'
                    when label in ('Spont. Tidal Volume', 'Tidal Volume (Spont)') then 'Tidal Volume (spontaneous)'
                    when label = 'Tidal Volume (set)' then 'Tidal Volume (Set)'
                    when label = 'Tidal Volume (Obser)' then 'Tidal Volume (observed)'
                else label end as label, 
                target_time,
                round(CAST((extract(epoch from target_time - charttime) / 3600.0) as numeric),2) as hours_from_charttime_to_targettime
                from chartevents as a 
                inner join d_items as b using (itemid)
                inner join (select identifier, target_time from model_a_cohort_updated) as c 
                    on a.subject_id||'-'||a.hadm_id = c.identifier
                where itemid in
                ( 448, 449, 450, 224687									-- minute volume
                , 654, 681, 682, 683, 684,224685,224684,224686			-- tidal volume
                , 535, 224695											-- Peak Insp. Pressure
                , 444, 224697 											-- Mean Airway Pressure
                , 1211, 1655, 2000, 224738								-- Inspiratory Time
                , 543, 224696 											-- PlateauPressure
                , 60, 505, 506, 686, 220339, 224700  					-- PEEP 
                , 618, 619, 220210, 224688, 224689, 224690              -- RR (Respiratory Rate)
                , 223835, 2981, 3420									-- FiO2 (Inspired O2 Fraction)
                ) 
                and subject_id||'-'||hadm_id in (select identifier from model_a_cohort_updated)
                and (extract(epoch from target_time - charttime)) > 0
                and valuenum is not null
                order by identifier, charttime
                )'''
                ]
    for command in commands:
        cur.execute(command)
    conn.commit()
    print("done creating respiratory_checks.")


def create_cohort_a_stays_info_limited_relevantcols(cur, conn):
    commands = [  # stays info for all icustays of the identifiers in the cohort:
        "DROP TABLE IF EXISTS cohort_a_stays_info",
        '''CREATE TABLE cohort_a_stays_info as (
                select a.subject_id||'-'||a.hadm_id as identifier, a.subject_id, a.hadm_id, icustay_id, 
                case when CAST((extract(epoch from intime - icu_time)) as numeric) between 0 and 60 then 1 else 0 end as icustayid_in_cohort,
                a.admittime as hospital_admittime, dischtime as hospital_dischtime, 
                round(CAST((extract(epoch from dischtime - a.admittime) / (3600.0*24) ) as numeric),2) as hospital_stay_len_days,
                hospital_expire_flag as died_in_hospital, p.expire_flag as patient_died, 
                case when (hospital_expire_flag = 1 and p.expire_flag = 1) then deathtime else p.dod end as deathtime,
                admission_type, admission_location, discharge_location as hosp_discharge_location, diagnosis as diagnosis_on_admission, 
                first_careunit as first_icu_careunit, last_careunit as last_icu_careunit, icu_time as cohort_icu_intime, intime as icu_intime, outtime as icu_outtime,
                los as len_of_icustay_days, 
                round(CAST((extract(epoch from c.intime - a.admittime) / 3600.0 ) as numeric),2) as hours_from_admittime_to_icu_intime,
                target_time, round(CAST((extract(epoch from target_time - a.admittime) / 3600.0 ) as numeric),2) as hours_from_admittime_to_targettime,
                round(CAST((extract(epoch from target_time - c.intime) / 3600.0 ) as numeric),2) as hours_from_icu_intime_to_targettime,
                round(CAST((extract(epoch from dischtime - target_time) / 3600.0 ) as numeric),2) as hours_from_targettime_to_discharge
                from admissions as a
                inner join model_a_cohort_updated as r on r.identifier = a.subject_id||'-'||a.hadm_id
                inner join patients as p on p.subject_id = a.subject_id
                inner join icustays as c on c.subject_id||'-'||c.hadm_id = a.subject_id||'-'||a.hadm_id
                order by identifier, icustay_id
                )''',
        # number of icu stays during the hospital admission:
        "DROP TABLE IF EXISTS num_of_icustays",
        '''CREATE TABLE num_of_icustays as (
                select identifier, count(*) as num_of_icustays
                from cohort_a_stays_info
                group by identifier
                order by identifier
                )''',
        # stays info only for icustays in which the target was taken, including number of icu stays during the hospital admission:
        "DROP TABLE IF EXISTS cohort_a_stays_info_limited",
        '''CREATE TABLE cohort_a_stays_info_limited as (
                with tmp as (
                select *
                from cohort_a_stays_info
                where icustayid_in_cohort = 1
                )
                select *
                from tmp
                inner join num_of_icustays
                using (identifier)
                )''',
        # cohort_a_stays_info_limited with the relevant columns for feature generation:
        "DROP TABLE IF EXISTS cohort_a_stays_info_limited_relevantcols",
        '''CREATE TABLE cohort_a_stays_info_limited_relevantcols as (
                select identifier, subject_id, hadm_id, hospital_stay_len_days, died_in_hospital, len_of_icustay_days,
                hours_from_admittime_to_icu_intime, hours_from_admittime_to_targettime, hours_from_icu_intime_to_targettime,
                hours_from_targettime_to_discharge, num_of_icustays
                from cohort_a_stays_info_limited
                )'''
    ]
    for command in commands:
        cur.execute(command)
    conn.commit()
    print("done creating cohort_a_stays_info_limited_relevantcols.")


def create_cohort_a_general_info(cur, conn):
    commands = [  # general info of the patients in cohort, some are textual
        "DROP TABLE IF EXISTS cohort_a_general_info",
        '''CREATE TABLE cohort_a_general_info as (
                select a.subject_id||'-'||a.hadm_id as identifier, a.subject_id, a.hadm_id,
                case when gender = 'M' then 0 when gender = 'F' then 1 end as gender,
                case when estimated_age>=300 then estimated_age-210 
                     else estimated_age end as age,
                insurance, language, religion, marital_status, ethnicity 
                from admissions as a
                inner join (select identifier, gender, estimated_age from relevant_events group by identifier, gender, estimated_age) as r 
                    on r.identifier = a.subject_id||'-'||a.hadm_id
                order by identifier
                )'''
    ]
    for command in commands:
        cur.execute(command)
    conn.commit()
    print("done creating cohort_a_general_info.")


def create_blood_cultures_table(cur, conn):
    commands = ["DROP TABLE IF EXISTS a_blood_cultures",
                '''CREATE TABLE a_blood_cultures as (
                select a.subject_id||'-'||a.hadm_id as identifier, a.subject_id, a.hadm_id, b.icu_time,
                case when a.charttime is null then a.chartdate else a.charttime end as charttime, target_time,
                case when charttime >= b.icu_time then 1 else 0 end as in_icu,
                case when org_name is null then 0 else 1 end as result, 
                case when org_name is null then 'no growth' else org_name end as org_name, 
                case when target_time - charttime >= '3 days' then 1 else 0 end as at_least_3_days_prior_targettime
                from microbiologyevents as a
                inner join model_a_cohort_updated as b on a.subject_id||'-'||a.hadm_id = b.identifier
                where spec_type_desc in ( 'BLOOD CULTURE','BLOOD CULTURE ( MYCO/F LYTIC BOTTLE)')
                and (extract(epoch from target_time - charttime)) >= 0
                group by identifier, case when a.charttime is null then a.chartdate else a.charttime end, 
                a.subject_id, a.hadm_id, b.icu_time, target_time, in_icu, result, org_name, at_least_3_days_prior_targettime 
                order by identifier, charttime
                )'''
                ]
    for command in commands:
        cur.execute(command)
    conn.commit()
    print("done creating a_blood_cultures.")


def create_cultures_table(cur, conn):
    commands = [  # urine, stool, and sputum cultures:
        "DROP TABLE IF EXISTS a_cultures",
        '''CREATE TABLE a_cultures as (
                with tmp as(
                select a.subject_id||'-'||a.hadm_id as identifier, a.subject_id, a.hadm_id, spec_type_desc as spec_type, 
                case when a.charttime is null then a.chartdate else a.charttime end as charttime,  
                case when charttime >= b.icu_time then 1 else 0 end as in_icu,
                case when org_name is null then 0 else 1 end as result, 
                case when org_name is null then 'no growth' else org_name end as org_name,
                target_time,
                case when target_time - charttime >= '3 days' then 1 else 0 end as at_least_3_days_prior_targettime
                from microbiologyevents as a
                inner join model_a_cohort_updated as b on a.subject_id||'-'||a.hadm_id = b.identifier
                where spec_type_desc in ('URINE','STOOL','SPUTUM')
                and (extract(epoch from target_time - charttime)) >= 0
                group by identifier, case when charttime is null then chartdate else charttime end, a.subject_id, a.hadm_id,
                spec_type_desc, in_icu, result, org_name, target_time, at_least_3_days_prior_targettime 
                order by identifier, charttime
                )
                select *, round(CAST((extract(epoch from target_time - charttime) / 3600.0) as numeric),2) as hours_from_charttime_to_targettime
                from tmp)'''
    ]
    for command in commands:
        cur.execute(command)
    conn.commit()
    print("done creating a_cultures.")


def create_lines_table(cur, conn):
    commands = ["DROP TABLE IF EXISTS catheter_table",
                '''CREATE TABLE catheter_table as (
                    select subject_id||'-'||hadm_id as identifier, subject_id, hadm_id ,icustay_id, starttime as start_date, endtime as end_date, 
                    round(CAST((extract(epoch from endtime - starttime) / 3600.0) as numeric),2) as duration_in_hours,
                    label as order_name, target_time,
                    round(CAST((extract(epoch from target_time - starttime) / 3600.0) as numeric),2) as hours_from_starttime_to_targettime
                    from procedureevents_mv as a
                    inner join D_ITEMS as b using (itemid) 
                    inner join (select identifier, target_time from model_a_cohort_updated) as d
                        on a.subject_id||'-'||a.hadm_id = d.identifier
                    where b.category in ('Access Lines - Invasive') 
                    and (extract(epoch from target_time - starttime)) > 0
                    order by identifier, starttime
                )''',
                # take only arterial lines, central venous lines, dialysis catheter, to use as features:
                "DROP TABLE IF EXISTS lines",
                '''CREATE TABLE lines as (
                select *, 
                case when order_name = 'Arterial Line' then 'Arterial Line'
                    when order_name = 'Dialysis Catheter' then 'Dialysis Catheter'
                    else 'Central Venous Line'
                    end as label
                from catheter_table	
                order by identifier, start_date
                )'''
                ]
    for command in commands:
        cur.execute(command)
    conn.commit()
    print("done creating lines.")

#########################################################################################

def clean_input_liquid_table(cur, conn):
    commands = ["DROP TABLE IF EXISTS input_liquid_clean",
                '''CREATE TABLE input_liquid_clean as (
                select identifier, start_date, end_date, DATE_PART('day', end_date - start_date)+1 as duration_days,
                order_name, category, amount, originalamount, originalrate, target_time, hours_from_starttime_to_targettime
                from input_liquid_table
                where originalamount <= 10000 and originalamount > 0
                )'''
                ]
    for command in commands:
        cur.execute(command)
    conn.commit()
    print("done creating input_liquid_clean.")


def clean_bmi(cur, conn):
    commands = ["DROP TABLE IF EXISTS bmi_clean",
                '''CREATE TABLE bmi_clean as (
                select identifier, subject_id, hadm_id, 
                case when admit_weight_kg < 0 or admit_weight_kg > 300 then null
                    else admit_weight_kg
                    end as admissionweight, 
                case when height_cm < 100 or height_cm > 240 then null
                    else height_cm
                    end as admissionheight
                from bmi
                )'''
                ]
    for command in commands:
        cur.execute(command)
    conn.commit()
    print("done creating bmi_clean.")


def clean_daily_weight(cur, conn):
    commands = ["DROP TABLE IF EXISTS daily_weight_clean",
                # remove measurements from the same day, keep only the first in every day, and clean data
                '''CREATE TABLE daily_weight_clean as (
                with tmp as(
                select *,
                case when date(charttime) = date(LAG(charttime, 1) OVER (partition by identifier order by charttime)) 
                    and abs((kg - LAG(kg, 1) OVER (partition by identifier order by charttime))) < 2 then 1 else 0 end as to_remove
                from daily_weight
                where kg > 0 and kg < 300 and kg is not null
                )
                select *
                from tmp
                where to_remove = 0
                )'''
                ]
    for command in commands:
        cur.execute(command)
    conn.commit()
    print("done creating daily_weight_clean.")


def clean_respiratory_checks(cur, conn):
    commands = ["DROP TABLE IF EXISTS respiratory_checks_clean",
                '''CREATE TABLE respiratory_checks_clean as (
                select *
                from respiratory_checks
                where (label in ('Mean Airway Pressure', 'Peak Insp. Pressure', 'Plateau Pressure') 
                or (label = 'FIO2' and value >=0 and value <=100)
                or (label like '%Minute Volume%' and value >=0 and value <=50)
                or (label like '%Tidal Volume%' and value >=0 and value <=1200)
                or (label like '%PEEP%' and value >=0 and value <=30)
                or (label = 'Inspiratory Time' and value >=0 and value <=10)
                or (label like '%Respiratory Rate%' and value >=0 and value <=65))
                and label != 'Respiratory Rate (spontaneous)' --remove it
                )'''
                ]
    for command in commands:
        cur.execute(command)
    conn.commit()
    print("done creating respiratory_checks_clean.")

#########################################################################################

# the commented out lines are for creating features that were not selected in the 40 features selected 
# in feature selection, hence we don't create them for the test data, to shorten running time 

def create_all_tables_model_a(file_path, conn, cur):
    create_cohort_table(cur, file_path, conn)
    create_relevent_events_table(cur, conn)
    create_updated_cohort(cur, conn)
    create_antibiotics_table(cur, conn)
    create_pressor_sedatives_meds_table(cur, conn)
    #create_output_liquid_table(cur, conn)
    #create_input_liquid_table(cur, conn)
    create_gcs_score_table(cur, conn)
    #create_bmi_table(cur, conn)
    #create_daily_weight_table(cur, conn)
    create_respiratory_checks_table(cur, conn)
    create_cohort_a_stays_info_limited_relevantcols(cur, conn)
    #create_cohort_a_general_info(cur, conn)
    create_blood_cultures_table(cur, conn)
    #create_cultures_table(cur, conn)
    create_lines_table(cur, conn)
    
def clean_tables(cur, conn):
    print("a- starting creating clean tables.")
    #clean_input_liquid_table(cur, conn)
    #clean_bmi(cur, conn)
    #clean_daily_weight(cur, conn)
    clean_respiratory_checks(cur, conn)
    
    sql = "select * from relevant_events"
    table = pd.io.sql.read_sql(sql, conn)
    clean_relevant_events(table, 'a')
    print("a- done creating clean tables.")

